# Tokenize and Gather Lemmas for Cleaned Reddit Comments

In [ ]:
! pip install spacy

In [ ]:
! pip install vaderSentiment

In [ ]:
import pickle
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import nltk
import pandas as pd
import spacy
import time

# NOTE: stopwords include pronouns! TODO: make custom stop words list?
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

* Modify list of Stop Words to remove words of interest

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize


stop_words = set(stopwords.words("english"))
keep_stop = set(['she', 'her', 'hers', 'herself'])
mod_stop_words = stop_words - keep_stop

In [ ]:
lmtzr = WordNetLemmatizer()

In [81]:
comments_corpus = pickle.load(open("reddit_full_cleaned.pickle","rb"))

In [ ]:
comments_corpus

* Turn tokenized comment into Lemmas

In [82]:
cleaned_comments = comments_corpus['cleaned_text']

lemmatized = [[lmtzr.lemmatize(word) for word in word_tokenize(s)]
              for s in cleaned_comments]

empty_list = list()
for lemmas in lemmatized:
    empty_list.append(' '.join(lemmas))
    

comments_corpus['lemmas'] = lemmatized
comments_corpus['lemmas_str'] = empty_list

In [ ]:
comments_corpus

* Use Lemmatized comment to detremine sentiment

In [83]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

# Negative
comments_corpus['neg'] = [analyzer.polarity_scores(v)['neg'] for v in comments_corpus['lemmas_str']]

In [84]:
# Compound
comments_corpus['compound'] = [analyzer.polarity_scores(v)['compound'] for v in comments_corpus['lemmas_str']]

In [85]:
# Positive
comments_corpus['pos'] = [analyzer.polarity_scores(v)['pos'] for v in comments_corpus['lemmas_str']]

In [86]:
comments_corpus

,subreddit,comment,time created,cleaned_text,lemmas,lemmas_str,neg,compound,pos
0,TheRedPill,Why the fuck would you be friends with an ex?,1451606442,fuck would friends ex,"[fuck, would, friend, ex]",fuck would friend ex,0.402,-0.0772,0.368
1,TheRedPill,Great story full of RP truths. Thanks. Conside...,1451606479,great story full rp truths thanks consider mak...,"[great, story, full, rp, truth, thanks, consid...",great story full rp truth thanks consider maki...,0.000,0.8519,0.482
2,TheRedPill,Lets suss this out. \n\nFor one; she's either ...,1451606497,lets suss one shes either got high paying job ...,"[let, sus, one, shes, either, got, high, payin...",let sus one shes either got high paying job cl...,0.076,0.0790,0.068
3,TheRedPill,Sounds like you may need some more treatment f...,1451606532,sounds like may need treatment anxiety crucial...,"[sound, like, may, need, treatment, anxiety, c...",sound like may need treatment anxiety crucial ...,0.046,0.8020,0.243
4,TheRedPill,"hack her phone or fb, its the only way you'll ...",1451606539,hack her phone fb way youll truly know think h...,"[hack, her, phone, fb, way, youll, truly, know...",hack her phone fb way youll truly know think h...,0.000,0.6124,0.357
...,...,...,...,...,...,...,...,...,...
574679,technews,PLEASE! I GET LIKE 3Mb/s I NEEEED THIS,1483147728,please get like 3mbs neeeed,"[please, get, like, 3mbs, neeeed]",please get like 3mbs neeeed,0.000,0.5859,0.615
574680,technews,Took some screenshots if it gets removed avail...,1483189543,took screenshots gets removed available httpii...,"[took, screenshots, get, removed, available, h...",took screenshots get removed available httpiim...,0.000,0.0000,0.000
574681,technews,[removed],1483199435,removed,[removed],removed,0.000,0.0000,0.000
574682,technews,"This is the best tl;dr I could make, [original...",1483205392,best tldr could make originalhttpwwwreuterscom...,"[best, tldr, could, make, originalhttpwwwreute...",best tldr could make originalhttpwwwreuterscom...,0.000,0.9231,0.144


* Change utc time created to DateTime object

In [ ]:
def change_time(df):
    '''
    converts utc code to datetime object
    '''
    converted_times = []
    for time in df['time created']:
        converted_times.append(datetime.fromtimestamp(float(time), tz=timezone.utc))
        
    df['time created'] = converted_times
        

In [ ]:
change_time(comments_corpus)
comments_corpus

In [ ]:
with open('reddit_comments_with_sentiment.pickle', 'wb') as handle:
    a = pickle.dump(comments_corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('reddit_comments_with_sentiment.pickle', 'rb') as handle:
    b = pickle.load(handle)